**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy.utilities.lambdify import lambdify
from scipy.optimize import minimize

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
def plot_func(xx, yy, zz, X=None):
    plt.contour(xx, yy, zz, cmap='Spectral')
    # both axes at the same scale + create a legend
    plt.gca().set_aspect('equal')
    plt.xlabel('x'); plt.ylabel('y')
    plt.colorbar(label='z')
    
    if not X is None:
        plt.scatter(X[:, 0], X[:, 1])

## Optimalizácia pomocou balíčka `scipy`

Ďalej si ako príklad ukážeme, ako sa dá optimalizácia realizovať pomocou balíčka `scipy`. Tento balíček implementuje viacero pokročilých metód, vrátane metód druhého rádu. Ide o metódy, ktoré sú spravidla účinnejšie než metóda klesajúceho gradientu a jej nadstavby, ktorým sme sa doteraz venovali. Ich nevýhodou naopak je nízka škálovateľnosť: typicky sú nepoužiteľné pre problémy s väčším počtom parametrov (pri strojovom učení je podobný problém aj väčšími dátovými množinami).

### Definícia účelovej funkcie

Na začiatok si znovu zadefinujeme účelovú funkciu a určíme jej gradient.



In [ ]:
symx, symy = sp.symbols('x y')
symf = (5*symx)**2 + symy ** 2
f = lambdify((symx, symy), symf, "numpy")

sym_grad_f = sp.Matrix([symf]).jacobian([symx, symy])
grad_f = lambdify((symx, symy), sym_grad_f, "numpy")

Ako zvyčajne si zobrazíme aj vizualizáciu.



In [ ]:
xx, yy = np.mgrid[-10:10.2:0.2, -10:10.2:0.2]
zz = f(xx, yy)
plot_func(xx, yy, zz)

### Minimalizácia pomocou `scipy`

Ďalej už aplikujeme metódu `minimize`. Ako argumenty špecifikujeme:

* Minimalizovanú funkciu `fun`. Očakáva sa, že funkcia bude na vstupe akceptovať vektor, preto našu funkciu $f$ obaľujeme do lambda funkcie, ktorá jej vektor rozbalí na skalárne argumenty $x$ a $y$ pomocou operátora *.
* Počiatočný bod `x0` z ktorého minimalizácia začína.
* Metódu: tu sa dá vybrať viacero rôznych solverov.
* Gradient: označuje sa `jac`, pretože sa dá špecifikovať aj celá Jakobiho matica (pre vektorové funkcie).


In [ ]:
res = minimize(fun=lambda X: f(*X),
               x0=[-9, -8],
               method='L-BFGS-B',
               jac=lambda X: grad_f(*X))

Na výstupe funkcia navráti objekt obsahujúci jednak výsledný bod a jednak hodnotu účelovej funkcie v danom bode:



In [ ]:
print("The point: {}".format(res.x))
print("The value: {}".format(res.fun))

Podrobnejšiu dokumentáciu ku funkcii je možné nájsť tu:



In [ ]:
print(minimize.__doc__)

### Vizualizácia postupu minimalizácie

Ak si chceme (tak ako v predchádzajúcich prípadoch) vizualizovať nielen výsledok, ale aj postup minimalizácie, môžeme použiť ešte argument `callback`, z ktorého budeme každý nový bod pridávať do zoznamu `X`.



In [ ]:
X = [[-9, -8]]

res = minimize(fun=lambda X: f(*X),
               x0=X[0],
               method='L-BFGS-B',
               jac=lambda X: grad_f(*X),
               callback=X.append)

X = np.array(X)

Výsledná vizualizácia bude vyzerať takto:



In [ ]:
xx, yy = np.mgrid[-10:10.2:0.2, -10:10.2:0.2]
zz = f(xx, yy)
plot_func(xx, yy, zz, X)

### Minimalizácia funkcie bez udania gradientu

Funkciu `minimize` je možné zavolať aj bez udania gradientu (`jac`). Niektoré solvery gradient nepoužívajú. Pre ostatné je možné gradient odhadnúť aj numericky (perturbáciou vstupnej premennej). Numerický odhad gradientu je praktické robiť len ak vstup nemá príliš veľa rozmerov – inak je to neprijateľne výpočtovo náročné.



In [ ]:
X = [[-9, -8]]

res = minimize(fun=lambda X: f(*X),
               x0=X[0],
               method='L-BFGS-B',
               callback=X.append)

In [ ]:
print("The result: {}".format(res.x))
print("The function's value: {}".format(res.fun))

Výsledok minimalizácie môžete porovnať s tým predchádzajúcim. Je možné, že bude o niečo horší, pretože funkcia teraz nemá k dispozícii skutočný gradient.

